In [ ]:
import numpy as np
import os



data = np.load("../input/practice/224_data.npy")
labels = np.load("../input/practice/224_labels.npy")

print(data.shape)
print(labels.shape)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)
print(X_train.shape)
print(X_test.shape)

In [ ]:
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.layers import Dense, Dropout, GlobalAveragePooling2D
from keras.models import Model
from keras.callbacks import ModelCheckpoint


#IncRes.summary()

In [ ]:
IncRes= InceptionResNetV2(weights='imagenet',include_top=False,input_shape=(224,224,3))
IncRes = Model(input=IncRes.input,output=IncRes.get_layer('block17_9_ac').output)

x = IncRes.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024,activation='relu')(x)
x = Dropout(0.5)(x)
prediction = Dense(2, activation='softmax', name='prediction')(x)

from keras.optimizers import SGD
from keras.optimizers import RMSprop

model = Model(inputs=IncRes.input,outputs=prediction)

rms = RMSprop(lr=0.000001)
model.compile(optimizer=rms,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

hist = model.fit(X_train, y_train,
      batch_size=64,
      epochs=30,
      shuffle=True,
      validation_split=0.1,
      verbose=1)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score


#model.load_weights("weights.best.hdf5")
y_pred = model.predict(X_test)


print("Accuracy: ",accuracy_score(y_test.argmax(axis=1),y_pred.argmax(axis=1)))
print("F1_score: ",f1_score(y_test.argmax(axis=1),y_pred.argmax(axis=1)))
print("Recall: ",recall_score(y_test.argmax(axis=1),y_pred.argmax(axis=1)))
print("Confusion Matrix",confusion_matrix(y_test.argmax(axis=1),y_pred.argmax(axis=1)))

In [ ]:
import matplotlib.pyplot as plt

## Plot Accuracy History

plt.plot(hist.history['acc'])
plt.plot(hist.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='lower right')
plt.show()

In [ ]:
## Plot Loss History

plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()